In [4]:
# DWave Credentials
dwave_solver = "DW_2000Q_2_1"
api_key = 'DEV-3142f58be13a0aacd44657b15b8fb9832ba3bdac'

In [9]:
# Import correlation data
import scipy.io as sio

P_array = sio.loadmat('P_mat.mat').get('P_mat')
A_array = sio.loadmat('A_mat.mat').get('A_mat')
S_array = sio.loadmat('S_mat.mat').get('S_mat')

In [33]:
# Import graph node labels
node_labels = []
with open('CryptoSymbols.csv') as f:
    for line in f.readlines():
        line = line.rstrip()
        node_labels.append(line)

In [26]:
graph_250 = S_array[250,:,:]
graph_250.shape

(27, 27)

In [76]:
### Use NetworkX graphic package to create a signed network
import networkx as nx

G = nx.Graph()
G.add_nodes_from(node_labels)

In [77]:
# Add weighted edges to graph
for i in range(27):
    for j in range(i+1, 27):
        if graph_250[i,j] < 0:
            G.add_edge(node_labels[i], node_labels[j], sign=-1)
        elif graph_250[i,j] > 0:
            G.add_edge(node_labels[i], node_labels[j], sign=1)

In [ ]:
# Print the relationships assigned by the current code execution
print('Friendly relationships: ', [(x, y) for (x, y, sign) in G.edges(data='sign') if (sign == 1)])
print('Hostile relationships: ', [(x, y) for (x, y, sign) in G.edges(data='sign') if (sign == -1)])

In [78]:
# Select a D-Wave system and handle mapping from problem graph to sampler graph
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

sampler = EmbeddingComposite(DWaveSampler(solver=dwave_solver, token=api_key))

In [79]:
# Return a good partition (minimal structural imbalance) and its frustrated edges  
import dwave_networkx as dnx
imbalance, bicoloring = dnx.structural_imbalance(G, sampler)

# Mark the returned frustrated edges and node set (color) on the graph 
for edge in G.edges:
    G.edges[edge]['frustrated'] = edge in imbalance
for node in G.nodes:
    G.nodes[node]['color'] = bicoloring[node]

# Print the relationships for the returned partition
print('Yellow set: ', [node for (node, color) in bicoloring.items() if (color == 0)])
print('Blue set: ', [node for (node, color) in bicoloring.items() if (color == 1)])
print('\nFrustrated relationships: ', list(imbalance.keys()))

Yellow set:  ['XRP', 'LTC', 'XLM', 'USDT', 'LSK', 'STEEM', 'SC', 'XVG', 'BTS', 'WAVES', 'DOGE', 'DGB', 'FCT']
Blue set:  ['BTC', 'ETH', 'XMR', 'DASH', 'XEM', 'BCN', 'DCR', 'DGD', 'MAID', 'PIVX', 'RDD', 'MONA', 'SYS', 'NXT']

Frustrated relationships:  [('BTC', 'XVG'), ('ETH', 'DOGE'), ('XRP', 'XVG'), ('LTC', 'DGD'), ('XLM', 'DGD'), ('XMR', 'XVG'), ('XMR', 'WAVES'), ('XMR', 'DOGE'), ('XMR', 'FCT'), ('DASH', 'DOGE'), ('USDT', 'XVG'), ('USDT', 'DOGE'), ('USDT', 'RDD'), ('XEM', 'XVG'), ('BCN', 'SC'), ('BCN', 'XVG'), ('BCN', 'BTS'), ('BCN', 'WAVES'), ('BCN', 'DOGE'), ('BCN', 'DGB'), ('BCN', 'FCT'), ('LSK', 'DGD'), ('DCR', 'SC'), ('DCR', 'XVG'), ('DCR', 'BTS'), ('DCR', 'WAVES'), ('DCR', 'DOGE'), ('DCR', 'FCT'), ('SC', 'DGD'), ('SC', 'MAID'), ('SC', 'RDD'), ('SC', 'SYS'), ('SC', 'NXT'), ('XVG', 'DGD'), ('XVG', 'MAID'), ('XVG', 'RDD'), ('XVG', 'SYS'), ('XVG', 'NXT'), ('BTS', 'DGD'), ('BTS', 'MAID'), ('BTS', 'SYS'), ('BTS', 'NXT'), ('WAVES', 'DGD'), ('WAVES', 'MAID'), ('WAVES', 'RDD'), ('WAVES'